In [52]:
from pydantic import BaseModel, Field, field_validator
from typing import List
import polars as pl
import instructor
import llama_cpp
import string
import json
import re

Load model, paragraph and sentence dataframe

In [53]:
modelfile = "../../llms/Meta-Llama-3-8B-Instruct.Q4_K_M.gguf"
llama = llama_cpp.Llama(
        model_path=modelfile,
        n_gpu_layers=-1,
        chat_format="llama-3", 
        n_ctx=2048,
        logits_all=True,
        verbose=False,
        echo = False,
        temperature=0
    )  

create = instructor.patch(
        create=llama.create_chat_completion_openai_v1,
        mode=instructor.Mode.JSON_SCHEMA,
    )

In [54]:
pars_df = pl.read_csv("../../outputs/max_havelaar/max_havelaar_paragraphs.tsv", separator="\t")
sents_df = pl.read_csv("../../outputs/max_havelaar/max_havelaar_sentences.tsv", separator="\t")

Define function to split into clauses

In [55]:
prompt = "Geef me je professionele hulp met deze zin, het is noodzakelijk dat je niet antwoordt met iets meer of minder dan de bijzinnen en dat je geen enkel teken van de invoer verliest (do not split in words):"
classdescription = """Een lijst van alle grammaticale bijzinnen in de invoerzin, zonder wijziging of verlies van leestekens en woorden van die zin. Bijzinnen moeten precies zo worden weergegeven als ze in de zin staan."""

class ClauseList(BaseModel):
    clauselist:List[str] = Field(
        description=classdescription
    )
    @field_validator('clauselist')
    def replace_empty_list(cls, v):
        if not v:
            return ["ERROR IN SENTENCE SPLITTING OUTPUT"]
        return v

def clausify_sent(sentence):
    extraction = create(
        response_model=ClauseList,
            messages=[
            {
            "role": "system",
            "content": f"{prompt}",
            },
            {
            "role": "user",
            "content": f"{prompt} {sentence}",
            }
            ]
        )

    clause_list =  extraction.model_dump()["clauselist"]
    return clause_list

In [56]:
def save_clauses(clauselist,sentence_id,chapter_id,paragraph_id,sentence,outfilename):
    clauses_dict = {index: clause for index, clause in enumerate(clauselist)}
    outdict = {
                "CHAPTER_ID":chapter_id,
                "SENTENCE_ID":sentence_id,
                "PARAGRAPH_ID":paragraph_id,
                "SENTENCETEXT":sentence,
                "CLAUSES":clauses_dict
                }
    with open(outfilename, "w") as output_file:
        json.dump(outdict, output_file)

In [57]:
def validate_clauses_match_sentence(clauselist, sentence):

    # Remove all white spaces from the sentence and concatenated clauses
    sentence_no_space_punct = ''.join(c for c in sentence if c not in string.whitespace + string.punctuation)
    joined_clauses_no_space_punct = ''.join(c for c in ''.join(clauselist) if c not in string.whitespace + string.punctuation)

    if sentence_no_space_punct != joined_clauses_no_space_punct:
        return ["ERROR IN THE LLM OUTPUT"]
    return clauselist

Find clauses in a given sentence

In [59]:
id_to_split = 5 #id of the sentence to process
matching_sents = sents_df.filter(pl.col("sentence_id") == id_to_split)
sentence = matching_sents["sentences"].to_list()[0]

In [39]:
if "«" in sentence:
    sentence = re.sub("«","\"",sentence)
if "»" in sentence:
    sentence = re.sub("»","\"",sentence)
print(sentence)
clauselist = clausify_sent(sentence)
print(clauselist)

Het is myn gewoonte niet, romans te schryven, of zulke dingen, en het heeft dan ook lang geduurd, voor ik er toe overging een paar riem papier extra te bestellen, en het werk aantevangen, dat gy, lieve lezer, zoo-even in de hand hebt genomen, en dat ge lezen moet als ge makelaar in koffi zyt, of als ge wat anders zyt.
['Het is mijn gewoonte niet, romans te schryven, of zulke dingen,', 'en het heeft dan ook lang geduurd, voor ik er toe overging een paar riem papier extra te bestellen,', 'en het werk aantevangen,', 'dat gy, lieve lezer, zoo-even in de hand hebt genomen,', 'en dat ge lezen moet als ge makelaar in koffi zyt,', 'of als ge wat anders zyt.']


In [60]:
after_validation = validate_clauses_match_sentence(clauselist, sentence)
print(after_validation)

['Als ik in myn vak - ik ben makelaar in koffi, en woon op de Lauriergracht N 37 - aan een principaal - een principaal is iemand die koffi verkoopt - een opgave deed, waarin maar een klein gedeelte der onwaarheden voorkwam, die in gedichten en romans de hoofdzaak uitmaken, zou hy terstond Busselinck & Waterman nemen.']


In [61]:
if after_validation == clauselist:
    outfilename = f"../../outputs/max_havelaar/clauses/Sentence{id_to_split}.json"
    parid = matching_sents["paragraph_id"].to_list()[0]
    chid = matching_sents["chapter_id"].to_list()[0]
    save_clauses(clauselist,id_to_split,chid,parid,sentence,outfilename)
    print("Saved!")

Saved!


Define functions to correct clauses  
e.g., if punctuation has been lost, decide if it's ok for your use case  
or you want to reintroduce it

In [12]:
def apply_correction(clauselist,sentence):
    corrected = []
    return corrected

In [ ]:
if not after_validation == clauselist:
    new_clauselist = apply_correction(clauselist,sentence)
    print(new_clauselist)
    #for x in new_clauselist:
    #    print(x)
    after_validation = validate_clauses_match_sentence(new_clauselist, sentence)
    print(after_validation)

In [ ]:
# Decide if you want to save
outfilename = f"../../outputs/max_havelaar/clauses/Sentence{id_to_split}.json"
print(outfilename)
parid = matching_sents["paragraph_id"].to_list()[0]
chid = matching_sents["chapter_id"].to_list()[0]
save_clauses(after_validation,id_to_split,parid,sentence,outfilename)

Once all clauses have been found, do a final check running
#$python3 scripts/preprocess/preprocess_book.py --clauses --retry